In [5]:
!pip install sentence_transformers

Looking in indexes: https://mirrors.tencent.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 6.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 12.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 41.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 9.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.3 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 9.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 9.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
%pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://mirrors.tencent.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 35.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import re

text = """
第一条　起草单位起草的政策措施，不得含有下列限制或者变相限制市场准入和退出的内容：
　　（一）对市场准入负面清单以外的行业、领域、业务等违法设置审批程序；
　　（二）违法设置或者授予特许经营权；
　　（三）限定经营、购买或者使用特定经营者提供的商品或者服务（以下统称商品）；
　　（四）设置不合理或者歧视性的准入、退出条件；
　　（五）其他限制或者变相限制市场准入和退出的内容。
第二条　起草单位起草的政策措施，不得含有下列限制商品、要素自由流动的内容：
　　（一）限制外地或者进口商品、要素进入本地市场，或者阻碍本地经营者迁出，商品、要素输出；
　　（二）排斥、限制、强制或者变相强制外地经营者在本地投资经营或者设立分支机构；
　　（三）排斥、限制或者变相限制外地经营者参加本地政府采购、招标投标；
　　（四）对外地或者进口商品、要素设置歧视性收费项目、收费标准、价格或者补贴；
　　（五）在资质标准、监管执法等方面对外地经营者在本地投资经营设置歧视性要求；
　　（六）其他限制商品、要素自由流动的内容。
第三条　起草单位起草的政策措施，没有法律、行政法规依据或者未经国务院批准，不得含有下列影响生产经营成本的内容：
　　（一）给予特定经营者税收优惠；
　　（二）给予特定经营者选择性、差异化的财政奖励或者补贴；
　　（三）给予特定经营者要素获取、行政事业性收费、政府性基金、社会保险费等方面的优惠；
　　（四）其他影响生产经营成本的内容。
第四条　起草单位起草的政策措施，不得含有下列影响生产经营行为的内容：
　　（一）强制或者变相强制经营者实施垄断行为，或者为经营者实施垄断行为提供便利条件；
　　（二）超越法定权限制定政府指导价、政府定价，为特定经营者提供优惠价格；
　　（三）违法干预实行市场调节价的商品、要素的价格水平；
　　（四）其他影响生产经营行为的内容。
第五条　起草单位起草的政策措施，具有或者可能具有排除、限制竞争效果，但符合下列情形之一，且没有对公平竞争影响更小的替代方案，并能够确定合理的实施期限或者终止条件的，可以出台：
　　（一）为维护国家安全和发展利益的；
　　（二）为促进科学技术进步、增强国家自主创新能力的；
　　（三）为实现节约能源、保护环境、救灾救助等社会公共利益的；
　　（四）法律、行政法规规定的其他情形。
"""  # 此处插入用户提供的文本内容

# 解析条款和子项
clauses = []
current_clause = None

for line in text.strip().split('\n'):
    if re.match(r'第.+条', line):
        current_clause = {
            "clause_id": re.search(r'第(.*?)条', line).group(1),
            "content": []
        }
        clauses.append(current_clause)
    elif re.match(r'　　（[一二三四五六]）', line):
        sub_item = re.sub(r'^　　（[一二三四五六]）', '', line).strip()
        current_clause["content"].append(sub_item)

In [7]:
from sentence_transformers import SentenceTransformer

# 加载中文预训练模型
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# 提取所有子项文本
texts = []
for clause in clauses:
    for sub in clause["content"]:
        texts.append(sub)

# 生成向量
embeddings = model.encode(texts)

/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [11]:
import faiss
import numpy as np

# 转换为FAISS需要的格式
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings.astype(np.float32))

# 保存索引（可选）
faiss.write_index(index, "legal_clauses_faiss.index")

In [12]:
# 输入查询语句
query = "限制外地企业参与招标"
query_embedding = model.encode([query])

# 搜索最相似的5个子项
k = 5
distances, indices = index.search(query_embedding.astype(np.float32), k)

# 输出结果
for idx in indices[0]:
    print(f"匹配子项：{texts[idx]}")
    print(f"所属条款：第{clauses[[i for i, c in enumerate(clauses) if idx < sum(len(c['content']) for c in clauses[:i+1])][0]]['clause_id']}条\n")

匹配子项：排斥、限制或者变相限制外地经营者参加本地政府采购、招标投标；
所属条款：第二条

匹配子项：对市场准入负面清单以外的行业、领域、业务等违法设置审批程序；
所属条款：第一条

匹配子项：限制外地或者进口商品、要素进入本地市场，或者阻碍本地经营者迁出，商品、要素输出；
所属条款：第二条

匹配子项：其他限制或者变相限制市场准入和退出的内容。
所属条款：第一条

匹配子项：排斥、限制、强制或者变相强制外地经营者在本地投资经营或者设立分支机构；
所属条款：第二条

